In [2]:
from datasets import load_dataset
import os

ds = load_dataset("hotchpotch/JQaRA", split="test")

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 最初は負荷軽減のため N 件だけ
N = int(os.getenv("LIMIT", "5000"))
subset = ds.select(range(min(N, len(ds))))

In [4]:
import weaviate

client = weaviate.connect_to_local()


In [5]:
CLASS_NAME="jqara"

if client.collections.exists(CLASS_NAME):
    client.collections.delete(CLASS_NAME)

In [6]:
from weaviate.classes.config import Configure, Property, DataType

OLLAMA_ENDPOINT="http://host.docker.internal:11434"
EMBED_MODEL="nomic-embed-text"
GEN_MODEL="llama3.2"

client.collections.create(
    name=CLASS_NAME,
    vector_config=Configure.Vectors.text2vec_ollama(
        api_endpoint=OLLAMA_ENDPOINT,
        model=EMBED_MODEL,
        # 必要なら instruction, options なども指定可
    ),
    generative_config=Configure.Generative.ollama(
        api_endpoint=OLLAMA_ENDPOINT,
        model=GEN_MODEL,
        # options={"temperature": 0.2} など
    ),
    properties=[
        Property(name="q_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)


In [7]:
from tqdm import tqdm

col = client.collections.get(CLASS_NAME)

with col.batch.dynamic() as batch:
    for row in tqdm(subset):
        batch.add_object(
            properties={
                "q_id": row.get("q_id"),
                "title": row.get("title"),
                "text": row.get("text"),
            }
        )

print(f"Imported {subset.num_rows} objects into {CLASS_NAME}")

100%|██████████| 5000/5000 [00:54<00:00, 91.95it/s] 


Imported 5000 objects into jqara


In [9]:
from weaviate.classes.query import MetadataQuery

query = subset[0]["question"]
res = col.query.near_text(
    query=query, 
    limit=5,
    return_metadata=MetadataQuery(distance=True, certainty=True, score=True)
)
print(f"\nquery: {query}")
print("\n=== Vector search results ===")
for o in res.objects:
    title = (o.properties or {}).get("title")
    text = (o.properties or {}).get("text") or ""
    # near_text なら通常 distance / certainty が入る（要求した場合）
    dist = getattr(o.metadata, "distance", None)
    cert = getattr(o.metadata, "certainty", None)
    score = getattr(o.metadata, "score", None)  # ハイブリッド/BM25で付くことがある

    # どれが取得できたかで柔軟に表示
    if dist is not None:
        metric = f"distance={dist:.4f}"
    elif score is not None:
        metric = f"score={score:.4f}"
    elif cert is not None:
        metric = f"certainty={cert:.4f}"
    else:
        metric = "(no metric)"

    print(f"- {metric}  title: {title}")
    print(f"  text: {text[:120]}...")
    print(f"  q_id: {(o.properties or {}).get('q_id')}, answers: {(o.properties or {}).get('answers')}\n")



query: 摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?

=== Vector search results ===
- distance=0.2617  title: 停止 (原子炉)
  text: 典型的な研究用原子炉の停止余裕は、キセノンのない冷たい状態で定義される。この条件にあてはまるとき、停止余裕は単に制御棒の効果の合計からcore excessを差し引くことで計算できる。最小停止余裕は停止余裕と類似の方法により計算される。異な...
  q_id: QA20CAPR-1004, answers: None

- distance=0.3221  title: 夜明け前
  text: 馬籠で造り酒屋を営んでいた大黒屋(作中の伏見屋)の当主、大脇兵衛門信興(明治3年没)は40年以上にわたる日記帳を書き残した。明治時代の大脇家は木曽一番の地主だったという。藤村は大脇家から日記帳を借り出し、第一部の参考資料とした。藤村が作成し...
  q_id: QA20CAPR-1014, answers: None

- distance=0.3274  title: 下原刑場
  text: 1868年(明治元年)の明治天皇の氷川神社行幸の際に、地元から刑場廃止の嘆願書が出され、廃止に至った。跡地は片倉工業の製糸所として利用された。1992年(平成8年)に工場が廃止されると隣接する大宮操車場跡地の開発であるさいたま新都心計画の一...
  q_id: QA20CAPR-1083, answers: None

- distance=0.3291  title: 椰子の実
  text: 「椰子の実」(やしのみ)は、島崎藤村が明治時代に執筆した詩。昭和に入って曲が付けられた。1900年(明治33年)6月の雑誌『新小説』に「海草」という総題で発表された誌の一遍で、1901年(明治34年)8月に刊行された詩集「落梅集」に収録され...
  q_id: QA20CAPR-1014, answers: None

- distance=0.3296  title: 孫子 (書物)
  text: この嚆矢となるものは1626年に出版された林羅山の『孫子諺解』であり、以降代表的なものだけでも山鹿素行『孫子諺義』(167

In [10]:

from weaviate.classes.generate import GenerativeConfig

response = col.generate.near_text(
    query=query,
    limit=2,
    grouped_task="質問に日本語で回答してください。",
    generative_provider=GenerativeConfig.ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="llama3.2"
    )
)

In [11]:
print(f"Grouped task result: {response.generative.text}")


Grouped task result: 質問に日本語で回答します。

原子炉の停止余裕は、キセノンのない冷たい状態（通常は100°C未満）で定義される。この条件を満たすと、停止余裕は制御棒の効果の合計からコアエクシースを差し引いて計算できる。最小停止余裕は同じ方法を使って計算されるが、一部の制御棒を除外して計算されることになっている。

この定義により、有効な制御棒がコアから脱落しても原子炉が安全に停止し続けることができるように設計することができる。さらに、この条件では冷却システムが常圧で華氏200度未満のとき、原子炉は「冷温停止」と呼ばれる。これは気密でなくても軽水炉における冷却・減速材である水は沸騰しないからである。

また、この質問には、夜明け前というタイトルが付けられています。
